In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os
import xarray as xr
from cycler import cycler
import matplotlib.patches as mpatches

In [3]:
scenarios = ['ssp126','ssp245','ssp370','ssp585']

modelnames = ['BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 
                  'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

basins = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202', 'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 'BAKER':'3404',
            'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232', 'COLORADO':'3212', 
            'CHICO':'3209', 'TORNEALVEN':'6255', 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 
            'KUBAN':'6223', 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
            'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101', 'CLUTHA':'5406', 'YUKON':'4435', 'TAKU':'4431', 
             'SUSITNA':'4430','STIKINE':'4428', 'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
            'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408', 'COLUMBIA':'4406', 'ALSEK':'4401', 'NELSON':'4125', 
              'MACKENZIE':'4123','COLVILLE':'4110', 'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 
              'TALAS':'2913', 'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 
              'YELLOW RIVER':'2434', 'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 
              'INDUS':'2309', 'GANGES':'2306','BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103', 'YANGTZE' : '2433'}

regions = ['RGI 01', 'RGI 02', 'RGI 06', 'RGI 08', 'RGI 10', 'RGI 11', 'RGI 12', 
           'RGI 13', 'RGI 15', 'RGI 16', 'RGI 17', 'RGI 18']

gmodels = ['GloGEM', 'OGGM', 'PyGEM']

fpath0 = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/'

In [4]:
# Create new index using pandas date_range function
start_date = datetime.date(2000, 1, 1)
end_date = datetime.date(2100, 12, 1)
new_indices = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

In [5]:
#Importing runoff data
all_rf_data = {}
all_rf_data_annual = {}

for r, region in enumerate(regions):
    fpath = fpath0 + 'Runoff/' + region + '/'
    
    for s, SSP in enumerate(scenarios):
        if SSP not in all_rf_data:
            all_rf_data[SSP] = {}
            all_rf_data_annual[SSP] = {}
        
        for b, basin in enumerate(basins):
            if basin not in all_rf_data[SSP]:
                all_rf_data[SSP][basin] = {}
                all_rf_data_annual[SSP][basin] = {}
            
            for m, GCM in enumerate(modelnames):
                fname = f"runoff_{GCM}_{SSP}_{basin}.csv"
                file_path = os.path.join(fpath, fname)  # Construct the full file path

                if os.path.exists(file_path):
                    try:
                        temp_df = pd.read_csv(file_path, index_col=0)
                        temp_df.index = new_indices
                        temp_df.index = pd.to_datetime(new_indices)
                        
                        all_rf_data[SSP][basin][GCM] = temp_df
                        all_rf_data_annual[SSP][basin][GCM] = temp_df.resample('A').sum()
                    except FileNotFoundError:
                        continue

In [9]:
#Determining the year of peak water
from scipy.signal import savgol_filter

window_length = 15
polyorder = 5

peak_water_year = {}
smoothed_runoff = {}
for s, SSP in enumerate(scenarios):
    peak_water_year[SSP] = {}
    smoothed_runoff[SSP] = {}
    for b, basin in enumerate(basins):
        peak_water_year[SSP][basin] = {}
        smoothed_runoff[SSP][basin] = {}
        for m, GCM in enumerate(modelnames):
            df = all_rf_data_annual[SSP][basin][GCM]
            smoothed_runoff[SSP][basin][GCM] = df[0:100].apply(lambda col: savgol_filter(col, window_length, polyorder))
            #smoothed_runoff[SSP][basin][GCM]['OGGM'] = df[0:100].apply(lambda col: savgol_filter(col, window_length, polyorder))
            peak_water_year[SSP][basin][GCM] = smoothed_runoff[SSP][basin][GCM].idxmax()

In [10]:
#Creating time values and color schemes
yrs = np.arange(2000, 2100)
yrs_dt = pd.to_datetime([str(y) for y in yrs])

colorschemes = {}

colors_glo =  plt.colormaps['Greens']
line_colors_glo = colors_glo(np.linspace(0.2, 0.6, num = 12))
glo_cycler = cycler(color = line_colors_glo)
colorschemes['GloGEM'] = glo_cycler

colors_OG =  plt.colormaps['Blues']
line_colors_OG = colors_OG(np.linspace(0.2, 0.6,num = 12))
OG_cycler = cycler(color = line_colors_OG)
colorschemes['OGGM'] = OG_cycler

colors_py =  plt.colormaps['Purples']
line_colors_py = colors_py(np.linspace(0.2, 0.6,num = 12))
py_cycler = cycler(color = line_colors_py)
colorschemes['PyGEM'] = py_cycler

colors = ['darkgreen', 'royalblue', 'purple']
fill_colors = ['green', 'dodgerblue', 'purple']

In [15]:
#Examing our smoothed datasets and the assigned peak water times
num_rows = 25
num_cols = 3

for s, SSP in enumerate(scenarios):
    for m, GCM in enumerate(modelnames):
        
        figy, axy = plt.subplots(num_rows, num_cols, figsize=(10, 0.8*len(basins)))
        row_max_values = []
        r = -1
        c = 0
        
        for b, basin in enumerate(basins):
            if c % 3 == 0:
                r += 1  
            c = (c + 1) % num_cols
            for g, gmodel in enumerate(gmodels):
                axy[r,c].plot(yrs_dt, smoothed_runoff[SSP][basin][GCM][gmodel], label = gmodel, color = colors[g])
                axy[r,c].plot(peak_water_year[SSP][basin][GCM][gmodel], smoothed_runoff[SSP][basin][GCM][gmodel].loc[peak_water_year[SSP][basin][GCM][gmodel]], marker = 'o', color = fill_colors[g])
                
            #Adding in text of which basin
            row_max = max(axy[r, c].get_ylim())
            row_max_values.append(row_max)
            
            y_coord = row_max_values[b] * 0.92
            axy[r,c].text(pd.to_datetime('2038-01-01'), y_coord, basin)
                              
        green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
        blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
        purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
        
        axy[0,0].legend(handles=[green_patch, blue_patch, purple_patch], bbox_to_anchor=(2.46, (0.0175*len(basins))), ncol=3)
        
        plt.suptitle('Smoothed Runoff Projections and Calculated Peak Water Times for all Basins', x=0.52, y=0.892)
        
        # Save the plot to your desktop
        desktop_path = os.path.expanduser("~/Desktop/Lizz Research/Figures/Peak Water/")  # Get the path to your desktop
        file_name = f"PeakWater{GCM}_{SSP}.png"  # Name of the file to save
        
        
        # Combine the desktop path and file name to create the full file path
        save_path = os.path.join(desktop_path, file_name)
        
        # Save the plot as an image
        plt.savefig(save_path, dpi=300, bbox_inches='tight')

        plt.close(figy)

In [275]:
out_df = {}
for s, SSP in enumerate(scenarios):
    out_df[SSP] = {}
    for m, model in enumerate(modelnames):
        basin_data_list = []  # Create a list to store data for all basins
        for b, basin in enumerate(basins):
            glo_values = peak_water_year[SSP][basin][model]['GloGEM']
            pygem_values = peak_water_year[SSP][basin][model]['PyGEM']
            oggm_values = peak_water_year[SSP][basin][model]['OGGM']

            # Create a dictionary with values for the current basin
            data = {
                'GloGEM': glo_values,
                'OGGM': oggm_values,
                'PyGEM': pygem_values,
            }
            
            basin_data_list.append(data)  # Append data for the current basin
        
        # Create a DataFrame from the list of basin data
        out_df[SSP][model] = pd.DataFrame(basin_data_list, index=basins)  # Set the index as 'Basin'

In [278]:
# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Peak Water/'

for SSP in out_df:
    for GCM in out_df[SSP]:
        fname = f"PeakWaterYr{GCM}_{SSP}.csv"

        # Define the full path of the output file
        output_path = os.path.join(output_dir, fname)

        # Save the DataFrame as CSV
        out_df[SSP][GCM].to_csv(output_path, header=True, index=True)